# Step 1: Import Necessary Libraries

In [2]:
import geopandas as gpd
import json

# Step 2: Load Cleaned Data

In [3]:
# Load cleaned GeoJSON data
state_boundaries = gpd.read_file("state_geojson.geojson")
lga_boundaries = gpd.read_file("lga_geojson.geojson")
ward_boundaries = gpd.read_file("ward_geojson.geojson")

# Step 3: Create State-to-LGA and LGA-to-Ward Relationships

In [4]:
# Create mappings
state_to_lga = lga_boundaries.groupby("statename")["lganame"].apply(list).to_dict()
lga_to_ward = ward_boundaries.groupby("lganame")["wardname"].apply(list).to_dict()

# Generate HTML with full state names
state_options = "".join([f'<option value="{state}">{state}</option>' for state in state_to_lga.keys()])

# Step 4: Convert GeoDataFrames to GeoJSON

In [5]:
# Convert to GeoJSON format
state_geojson = state_boundaries.to_crs(epsg=4326).to_json()
lga_geojson = lga_boundaries.to_crs(epsg=4326).to_json()
ward_geojson = ward_boundaries.to_crs(epsg=4326).to_json()


# Step 5: Generate HTML for Interactive Map

Interactive map saved as 'interactive_map.html'


# Interactive map 2